In [ ]:
import utils

import numpy as np
from IPython.display import HTML, display, Markdown
import pandas as pd
import ot
import partial_gw as pgw
# import partial_gw_1 as pgw
import matplotlib.pyplot as plt

from subset_select_ipot_non_uniform_git import subset_select_ipot as ss_ipot
from subset_select_non_uniform_entropic_FISTA_git import subset_select_non_uniform_FISTA as ss

# from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score


In [2]:
# datasets = ['mushrooms']
# prior_lst = [0.518]
# prior_lst = [0.518, 0.786, 0.898, 0.167, 0.658, 0.394, 0.1, 0.1]
# ds_p = datasets[0]
# ds_ul = datasets[0]
# prior = prior_lst[0]


# n_unl = 800
# n_pos = 400
# nb_reps = 10
# nb_dummies = 10
# gamma_ss = 0.1
# c = 20  # subset selection scaling constant
# mu = np.ones(n_pos)/n_pos
# nu = np.ones(n_unl)/n_unl

# perfs_ds_pw, perfs_list_ds_pw, yu_list_pW, yhat_list_pW, nu_star_list_pW = pgw.compute_perf_emd(ds_p, ds_ul, n_pos, n_unl, prior, nb_reps, nb_dummies)


# PU Learning in Same Domain

In [3]:
from utils import make_data
from utils import draw_p_u_dataset_scar
# import pgw.compute_cost_matrices as cost_matrix

datasets = ['mushrooms', 'shuttle', 'pageblocks', 'usps', 'connect-4', 'spambase', 'mnist', 'mnist-colored']
# datasets = ['mushrooms']
prior_lst = [0.518, 0.786, 0.898, 0.167, 0.658, 0.394, 0.1, 0.1]

acc_pW_avg = []
acc_pGW_avg = []
acc_SS_avg = []

precision_pW_avg = []
precision_pGW_avg = []
precision_SS_avg = []

recall_pW_avg = []
recall_pGW_avg = []
recall_SS_avg = []

f1_scores_pW_avg = []
f1_scores_pGW_avg = []
f1_scores_SS_avg = []

roc_auc_pW_avg = []
roc_auc_pGW_avg = []
roc_auc_SS_avg = []

yu_dict_PW = {}
yhat_dict_PW = {}
yu_dict_PGW = {}
yhat_dict_PGW = {}
yu_dict_SS = {}
yhat_dict_SS = {}
nu_star_dict_PW = {}
nu_star_dict_PGW = {}
nu_star_dict_SS = {}


n_unl = 800
n_pos = 400
nb_reps = 10
nb_dummies = 10
gamma_ss = 0.1

cs = [n_unl / np.ceil(prior_lst[i]*n_unl) for i in range(len(prior_lst))]
# c = 20 # subset selection scaling constant
mu = np.ones(n_pos)/n_pos
nu = np.ones(n_unl)/n_unl

for i in range(len(datasets)):
    ds = datasets[i]
    c = cs[i]
    print(ds)
    if ds != "mnist-colored":
        ds_p = ds
        ds_ul = ds
    else:
        ds_p = 'mnist_color_change_p'
        ds_ul = 'mnist_color_change_u'

    prior = prior_lst[i]
    
    print("partial Wasserstein")
    perfs_ds_pw, perfs_list_ds_pw, yu_list_pW, yhat_list_pW, nu_star_list_pW = pgw.compute_perf_emd(ds_p, ds_ul, n_pos, n_unl, prior, nb_reps, nb_dummies)
    acc_pW_avg.append(perfs_ds_pw['emd_groups'])  
    
    print("partial Gromov-Wasserstein")  
    perfs_ds_pgw, perfs_list_pgw, yu_list_pGW, yhat_list_pGW, nu_star_list_pGW = pgw.compute_perf_pgw(ds_p, ds_ul, n_pos, n_unl, prior, nb_reps, nb_dummies)
    acc_pGW_avg.append(perfs_ds_pgw['pgw'])
    yu_dict_PW[i] = yu_list_pW
    nu_star_dict_PW[i] = nu_star_list_pW
    yu_dict_PGW[i] = yu_list_pGW
    nu_star_dict_PGW[i] = nu_star_list_pGW
    
    precision_pW = []
    recall_pW = []
    f1_scores_pW = []
    roc_auc_pW = []
    
    precision_pGW = []
    recall_pGW = []
    f1_scores_pGW = []
    roc_auc_pGW = []
    
    for reps in range(nb_reps):
        precision_pW_rep, recall_pW_rep, f1_pW_rep, _ = precision_recall_fscore_support(yu_list_pW[reps], yhat_list_pW[reps], pos_label=1, average='binary')
        precision_pW.append(precision_pW_rep)
        recall_pW.append(recall_pW_rep)
        f1_scores_pW.append(f1_pW_rep)
        roc_auc_pW.append(roc_auc_score(yu_list_pW[reps], nu_star_list_pW[reps]))
        
        
        precision_pGW_rep, recall_pGW_rep, f1_pGW_rep, _ = precision_recall_fscore_support(yu_list_pGW[reps], yhat_list_pGW[reps], pos_label=1, average='binary')
        precision_pGW.append(precision_pGW_rep)
        recall_pGW.append(recall_pGW_rep)
        f1_scores_pGW.append(f1_pGW_rep)
        roc_auc_pGW.append(roc_auc_score(yu_list_pGW[reps], nu_star_list_pGW[reps]))
        
    
    precision_pW_avg.append(np.mean(precision_pW))
    recall_pW_avg.append(np.mean(recall_pW))
    f1_scores_pW_avg.append(np.mean(f1_scores_pW))
    roc_auc_pW_avg.append(np.mean(roc_auc_pW))
    
    precision_pGW_avg.append(np.mean(precision_pGW))
    recall_pGW_avg.append(np.mean(recall_pGW))
    
    
    f1_scores_pGW_avg.append(np.mean(f1_scores_pGW))
    roc_auc_pGW_avg.append(np.mean(roc_auc_pGW))
    
    print("precision pW=", precision_pW_avg[-1])
    print("recall pW=", recall_pW_avg[-1])
    print("f1-score PW=", f1_scores_pW_avg[-1])
    print("roc-auc PW =", roc_auc_pW_avg[-1]) 
    print("acc_pW_avg", acc_pW_avg[-1])
    
    print("precision pGW = ", precision_pGW_avg[-1])
    print("recall pGW = ", recall_pGW_avg[-1])
    print("f1-score PGW = ", f1_scores_pGW_avg[-1])
    print("roc-auc PGW =", roc_auc_pGW_avg[-1]) 
    print("acc_pGW_avg", acc_pGW_avg[-1])
    

    acc_SS = []
    precision_SS = []
    recall_SS = []
    f1_scores_SS = []
    roc_auc_SS = []
    nu_star_lst = []
    yu_list_SS = []
    yhat_list_SS = []
    print("subset_slection")
    
    for reps in range(nb_reps):
        xp_df, xu_df, yu_df = draw_p_u_dataset_scar(ds_p, ds_ul, n_pos, n_unl, prior, reps)
        xp = xp_df.to_numpy()
        xu = xu_df.to_numpy()
        yu = yu_df.to_numpy()
        
        yu_list_SS.append(yu)

        costs, P, _, _ = ss_ipot(xp, xu, mu, nu, c, gamma=gamma_ss, max_outer_iter=200, max_inner_iter=20, return_map=True )
        nu_star = np.sum(P,axis=0)
        nu_star_lst.append(nu_star)
        nu_star_sorted_idx = np.argsort(-nu_star)
        # nu_start_selected_idx = nu_star_sorted_idx[0:int(prior_lst[i]*len(nu_star))]
        nu_start_selected_idx = nu_star_sorted_idx[0:int(np.sum(yu))]
        yu_hat = np.zeros_like(yu)
        yu_hat[nu_start_selected_idx] = 1
        acc_SS.append(np.mean(yu_hat == yu))
        yhat_list_SS.append(yu_hat)
        precision_SS_rep, recall_SS_rep, f1_score_SS_rep,  _ = precision_recall_fscore_support(yu, yu_hat, pos_label=1, average='binary')

        
        precision_SS.append(precision_SS_rep)
        recall_SS.append(recall_SS_rep)
        f1_scores_SS.append(f1_score_SS_rep)
        # roc_auc_SS.append(roc_auc_score(yu, yu_hat))
        roc_auc_SS.append(roc_auc_score(yu, nu_star))
    
    yu_dict_SS[i] = yu_list_SS
    yhat_dict_SS[i] = yhat_list_SS
        
    precision_SS_avg.append(np.mean(precision_SS))
    recall_SS_avg.append(np.mean(recall_SS))
    f1_scores_SS_avg.append(np.mean(f1_scores_SS))
    acc_SS_avg.append(np.mean(acc_SS))
    roc_auc_SS_avg.append(np.mean(roc_auc_SS))
    
    print("precision SS= ", precision_SS_avg[i])
    print("recall SS= ", recall_SS_avg[i])
    print("f1-score SS= ", f1_scores_SS_avg[i])
    print("roc auc SS=", roc_auc_SS_avg[i])
    print("acc_SS_avg=", acc_SS_avg[i])


mushrooms
partial Wasserstein
partial Gromov-Wasserstein
precision pW= 0.9531400966183575
recall pW= 0.9531400966183575
f1-score PW= 0.9531400966183575
roc-auc PW = 0.9657302070035794
acc_pW_avg 0.9514999999999999
precision pGW =  0.3608695652173913
recall pGW =  0.3608695652173913
f1-score PGW =  0.36086956521739133
roc-auc PGW = 0.3336058546719731
acc_pGW_avg 0.9484999999999999
subset_slection
precision SS=  0.9673913043478262
recall SS=  0.9673913043478262
f1-score SS=  0.9673913043478262
roc auc SS= 0.9882621836750018
acc_SS_avg= 0.9662499999999999
shuttle
partial Wasserstein
partial Gromov-Wasserstein
precision pW= 0.9689490445859873
recall pW= 0.9689490445859873
f1-score PW= 0.9689490445859873
roc-auc PW = 0.9318767589986668
acc_pW_avg 0.9512500000000002
precision pGW =  0.8407643312101911
recall pGW =  0.8407643312101911
f1-score PGW =  0.8407643312101911
roc-auc PGW = 0.6214894089764479
acc_pGW_avg 0.93625
subset_slection
precision SS=  0.9757961783439489
recall SS=  0.97579617

In [4]:
results_UCI = {'dataset': datasets,
               '$\pi$': prior_lst,
               'acc SS': acc_SS_avg,
               'prec SS': precision_SS_avg,
               'f1 SS ': f1_scores_SS_avg,
               'roc-auc SS': roc_auc_SS_avg,
               }
results_UCI = pd.DataFrame(data=results_UCI)
results_UCI

,dataset,$\pi$,acc SS,prec SS,f1 SS,roc-auc SS
0,mushrooms,0.518,0.96625,0.967391,0.967391,0.988262
1,shuttle,0.786,0.96200,0.975796,0.975796,0.971779
2,pageblocks,0.898,0.92400,0.957660,0.957660,0.851332
3,usps,0.167,0.98475,0.954135,0.954135,0.992713
4,connect-4,0.658,0.60725,0.701331,0.701331,0.587068
5,spambase,0.394,0.79275,0.736825,0.736825,0.836917
6,mnist,0.100,0.99175,0.958750,0.958750,0.997106
7,mnist-colored,0.100,0.91875,0.593750,0.593750,0.952113


In [5]:
results_UCI = {'dataset': datasets,
               '$\pi$': prior_lst,
               'acc PW': acc_pW_avg,
               'acc_PGW ': acc_pGW_avg,
               'acc SS': acc_SS_avg,
               'roc_auc PW': roc_auc_pW_avg,
               'roc_auc PGW': roc_auc_pGW_avg,
               'roc-auc SS': roc_auc_SS_avg
               
               }
results_UCI = pd.DataFrame(data=results_UCI)
results_UCI

,dataset,$\pi$,acc PW,acc_PGW,acc SS,roc_auc PW,roc_auc PGW,roc-auc SS
0,mushrooms,0.518,0.95150,0.94850,0.96625,0.965730,0.333606,0.988262
1,shuttle,0.786,0.95125,0.93625,0.96200,0.931877,0.621489,0.971779
2,pageblocks,0.898,0.91900,0.90350,0.92400,0.802465,0.720660,0.851332
3,usps,0.167,0.98275,0.95550,0.98475,0.981445,0.509594,0.992713
4,connect-4,0.658,0.60950,0.58050,0.60725,0.569227,0.512591,0.587068
5,spambase,0.394,0.78800,0.68400,0.79275,0.795228,0.583390,0.836917
6,mnist,0.100,0.99075,0.98225,0.99175,0.987444,0.763757,0.997106
7,mnist-colored,0.100,0.91575,0.96775,0.91875,0.818862,0.661891,0.952113


In [7]:
results_UCI = {'dataset':datasets, 
               '$\pi$': prior_lst,
               'acc pW': acc_pW_avg,
               'prec pW':precision_pW_avg,
               'f1 pW ': f1_scores_pW_avg,
               'roc-auc pW':roc_auc_pW_avg,
               'acc pGW': acc_pGW_avg,
               'prec pGW':precision_pGW_avg,
               'f1 pGW ': f1_scores_pGW_avg,
               'roc-auc pGW':roc_auc_pGW_avg,
               'acc SS': acc_SS_avg,
               'prec SS': precision_SS_avg,
               'f1 SS ': f1_scores_SS_avg,
               'roc-auc SS':roc_auc_SS_avg,
              }
results_UCI = pd.DataFrame(data=results_UCI)
results_UCI

,dataset,$\pi$,acc pW,prec pW,f1 pW,roc-auc pW,acc pGW,prec pGW,f1 pGW,roc-auc pGW,acc SS,prec SS,f1 SS,roc-auc SS
0,mushrooms,0.518,0.95175,0.953382,0.953382,0.969717,0.94850,0.237681,0.237681,0.202449,0.96625,0.967391,0.967391,0.988262
1,shuttle,0.786,0.95200,0.969427,0.969427,0.934746,0.93625,0.890924,0.890924,0.754279,0.96200,0.975796,0.975796,0.971779
2,pageblocks,0.898,0.91875,0.954735,0.954735,0.800212,0.90350,0.943872,0.943872,0.764272,0.92400,0.957660,0.957660,0.851332
3,usps,0.167,0.98275,0.948120,0.948120,0.985560,0.95550,0.134586,0.134586,0.467289,0.98475,0.954135,0.954135,0.992713
4,connect-4,0.658,0.60025,0.696008,0.696008,0.564149,0.58050,0.658365,0.658365,0.500248,0.60725,0.701331,0.701331,0.587068
5,spambase,0.394,0.78775,0.730476,0.730476,0.807205,0.68400,0.439365,0.439365,0.546846,0.79275,0.736825,0.736825,0.836917
6,mnist,0.100,0.99150,0.957500,0.957500,0.989003,0.98225,0.767500,0.767500,0.905652,0.99175,0.958750,0.958750,0.997106
7,mnist-colored,0.100,0.91575,0.578750,0.578750,0.816286,0.96775,0.732500,0.732500,0.900714,0.91875,0.593750,0.593750,0.952113


# cross domain PU Learning

In [6]:
# from utils import make_data
from utils import draw_p_u_dataset_scar


acc_pW_avg = []
acc_pGW_avg = []
acc_SS_avg = []

precision_pW_avg = []
precision_pGW_avg = []
precision_SS_avg = []

recall_pW_avg = []
recall_pGW_avg = []
recall_SS_avg = []

f1_scores_pW_avg = []
f1_scores_pGW_avg = []
f1_scores_SS_avg = []

roc_auc_pW_avg = []
roc_auc_pGW_avg = []
roc_auc_SS_avg = []

yu_dict_PW = {}
yhat_dict_PW = {}
yu_dict_PGW = {}
yhat_dict_PGW = {}
yu_dict_SS = {}
yhat_dict_SS = {}

nu_star_dict_PW = {}
nu_star_dict_PGW = {}
nu_star_dict_SS = {}


n_unl = 100
n_pos = 100
nb_reps = 10
nb_dummies = 10
prior = 0.1
gamma_ss = 0.1
c = n_unl / np.ceil(prior*n_unl)  # subset selection scaling constant
mu = np.ones(n_pos)/n_pos
nu = np.ones(n_unl)/n_unl

domain_u = ['surf_Caltech10', 'surf_amazon', 'surf_webcam', 'surf_dslr']
idx = 0

for ds_ul in domain_u:
    print('labeled='+'surf_Caltech10')
    print('unlabeled='+ ds_ul)

    prior = 0.1
    perfs_ds_pw, perfs_list_ds_pw, yu_list_pW, yhat_list_pW, nu_star_list_pW = pgw.compute_perf_emd('surf_Caltech10', ds_ul, n_pos, n_unl, prior, nb_reps, nb_dummies)
    acc_pW_avg.append(perfs_ds_pw['emd_groups'])
    perfs_ds_pgw, perfs_list_pgw, yu_list_pGW, yhat_list_pGW, nu_star_list_pGW = pgw.compute_perf_pgw('surf_Caltech10', ds_ul, n_pos, n_unl, prior, nb_reps, nb_dummies)
    acc_pGW_avg.append(perfs_ds_pgw['pgw'])
    
    yu_dict_PW[i] = yu_list_pW
    nu_star_dict_PW[i] = nu_star_list_pW
    yu_dict_PGW[i] = yu_list_pGW
    nu_star_dict_PGW[i] = nu_star_list_pGW

    precision_pW = []
    recall_pW = []
    f1_scores_pW = []
    roc_auc_pW = []

    precision_pGW = []
    recall_pGW = []
    f1_scores_pGW = []
    roc_auc_pGW = []

    for reps in range(nb_reps):
        precision_pW_rep, recall_pW_rep, f1_pW_rep, _ = precision_recall_fscore_support(yu_list_pW[reps], yhat_list_pW[reps], pos_label=1, average='binary')
        precision_pW.append(precision_pW_rep)
        recall_pW.append(recall_pW_rep)
        f1_scores_pW.append(f1_pW_rep)
        roc_auc_pW.append(roc_auc_score(yu_list_pW[reps], nu_star_list_pW[reps]))

        precision_pGW_rep, recall_pGW_rep, f1_pGW_rep, _ = precision_recall_fscore_support(yu_list_pGW[reps], yhat_list_pGW[reps], pos_label=1, average='binary')
        precision_pGW.append(precision_pGW_rep)
        recall_pGW.append(recall_pGW_rep)
        f1_scores_pGW.append(f1_pGW_rep)
        roc_auc_pGW.append(roc_auc_score( yu_list_pGW[reps], nu_star_list_pGW[reps]))

        # f_scores_pW.append(f1_score(yu_list_pW[reps], yhat_list_pW[reps], average='binary'))
        # f_scores_pGW.append(f1_score(yu_list_pGW[reps], yhat_list_pGW[reps], average='binary'))

    precision_pW_avg.append(np.mean(precision_pW))
    recall_pW_avg.append(np.mean(recall_pW))

    precision_pGW_avg.append(np.mean(precision_pGW))
    recall_pGW_avg.append(np.mean(recall_pGW))

    f1_scores_pW_avg.append(np.mean(f1_scores_pW))
    f1_scores_pGW_avg.append(np.mean(f1_scores_pGW))

    roc_auc_pW_avg.append(np.mean(roc_auc_pW))
    roc_auc_pGW_avg.append(np.mean(roc_auc_pGW))

    print("precision pW=", precision_pW_avg[idx])
    print("recall pW=", recall_pW_avg[idx])
    print("f1-score PW=", f1_scores_pW_avg[idx])
    print("roc-auc PW =", roc_auc_pW_avg[idx])
    print("acc_pW_avg", acc_pW_avg[idx])

    print("precision pGW = ", precision_pGW_avg[idx])
    print("recall pGW = ", recall_pGW_avg[idx])
    print("f1-score PGW = ", f1_scores_pGW_avg[idx])
    print("roc-auc PGW =", roc_auc_pGW_avg[idx])
    print("acc_pGW_avg", acc_pGW_avg[idx])

    acc_SS = []
    precision_SS = []
    recall_SS = []
    f1_scores_SS = []
    roc_auc_SS = []

    for reps in range(nb_reps):
        xp_df, xu_df, yu_df = draw_p_u_dataset_scar('surf_Caltech10', ds_ul, n_pos, n_unl, prior, reps)
        xp = xp_df.to_numpy()
        xu = xu_df.to_numpy()
        yu = yu_df.to_numpy()

        costs, P, _, _ = ss_ipot(xp, xu, mu, nu, c, gamma=gamma_ss, max_outer_iter=100, max_inner_iter=50, return_map=True)
        nu_star = np.sum(P, axis=0)
        nu_star_sorted_idx = np.argsort(-nu_star)
        # nu_start_selected_idx = nu_star_sorted_idx[0:int(prior_lst[i]*len(nu_star))]
        nu_start_selected_idx = nu_star_sorted_idx[0:int(np.sum(yu))]
        yu_hat = np.zeros_like(yu)
        yu_hat[nu_start_selected_idx] = 1
        acc_SS.append(np.mean(yu_hat == yu))
        # f1_scores_SS.append(f1_score(yu, yu_hat, average='binary'))
        precision_SS_rep, recall_SS_rep, f1_score_SS_rep,  _ = precision_recall_fscore_support(yu, yu_hat, pos_label=1, average='binary')

        precision_SS.append(precision_SS_rep)
        recall_SS.append(recall_SS_rep)
        f1_scores_SS.append(f1_score_SS_rep)
        roc_auc_SS.append(roc_auc_score(yu, yu_hat))

    precision_SS_avg.append(np.mean(precision_SS))
    recall_SS_avg.append(np.mean(recall_SS))
    f1_scores_SS_avg.append(np.mean(f1_scores_SS))
    acc_SS_avg.append(np.mean(acc_SS))
    roc_auc_SS_avg.append(np.mean(roc_auc_SS))
    
    

    print("precision SS= ", precision_SS_avg[idx])
    print("recall SS= ", recall_SS_avg[idx])
    print("f1-score SS= ", f1_scores_SS_avg[idx])
    print("roc auc SS=", roc_auc_SS_avg[idx])
    print("acc_SS_avg", acc_SS_avg[idx])
    
    idx = idx+1

domain_u = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam', 'decaf_dslr']

for ds_ul in domain_u:
    print('labeled='+'decaf_Caltech10')
    print('unlabeled=' + ds_ul)

    prior = 0.1
    perfs_ds_pw, perfs_list_ds_pw, yu_list_pW, yhat_list_pW, nu_star_list_pW  = pgw.compute_perf_emd('decaf_caltech', ds_ul, n_pos, n_unl, prior, nb_reps, nb_dummies)
    acc_pW_avg.append(perfs_ds_pw['emd_groups'])
    perfs_ds_pgw, perfs_list_pgw, yu_list_pGW, yhat_list_pGW, nu_star_list_pGW = pgw.compute_perf_pgw('decaf_caltech', ds_ul, n_pos, n_unl, prior, nb_reps, nb_dummies)
    acc_pGW_avg.append(perfs_ds_pgw['pgw'])
    
    yu_dict_PW[i] = yu_list_pW
    nu_star_dict_PW[i] = nu_star_list_pW
    yu_dict_PGW[i] = yu_list_pGW
    nu_star_dict_PGW[i] = nu_star_list_pGW

    precision_pW = []
    recall_pW = []
    f1_scores_pW = []
    roc_auc_pW = []

    precision_pGW = []
    recall_pGW = []
    f1_scores_pGW = []
    roc_auc_pGW = []



    for reps in range(nb_reps):
        precision_pW_rep, recall_pW_rep, f1_pW_rep, _ = precision_recall_fscore_support(yu_list_pW[reps], yhat_list_pW[reps], pos_label=1, average='binary')
        precision_pW.append(precision_pW_rep)
        recall_pW.append(recall_pW_rep)
        f1_scores_pW.append(f1_pW_rep)
        roc_auc_pW.append(roc_auc_score(yu_list_pW[reps], nu_star_list_pW[reps]))

        precision_pGW_rep, recall_pGW_rep, f1_pGW_rep, _ = precision_recall_fscore_support(yu_list_pGW[reps], yhat_list_pGW[reps], pos_label=1, average='binary')
        precision_pGW.append(precision_pGW_rep)
        recall_pGW.append(recall_pGW_rep)
        f1_scores_pGW.append(f1_pGW_rep)
        roc_auc_pGW.append(roc_auc_score(yu_list_pGW[reps], nu_star_list_pGW[reps]))


    precision_pW_avg.append(np.mean(precision_pW))
    recall_pW_avg.append(np.mean(recall_pW))

    precision_pGW_avg.append(np.mean(precision_pGW))
    recall_pGW_avg.append(np.mean(recall_pGW))

    f1_scores_pW_avg.append(np.mean(f1_scores_pW))
    f1_scores_pGW_avg.append(np.mean(f1_scores_pGW))

    roc_auc_pW_avg.append(np.mean(roc_auc_pW))
    roc_auc_pGW_avg.append(np.mean(roc_auc_pGW))

    print("precision pW=", precision_pW_avg[idx])
    print("recall pW=", recall_pW_avg[idx])
    print("f1-score PW=", f1_scores_pW_avg[idx])
    print("roc-auc PW =", roc_auc_pW_avg[idx])
    print("acc_pW_avg", acc_pW_avg[idx])

    print("precision pGW = ", precision_pGW_avg[idx])
    print("recall pGW = ", recall_pGW_avg[idx])
    print("f1-score PGW = ", f1_scores_pGW_avg[idx])
    print("roc-auc PGW =", roc_auc_pGW_avg[idx])
    print("acc_pGW_avg", acc_pGW_avg[idx])

    acc_SS = []
    precision_SS = []
    recall_SS = []
    f1_scores_SS = []
    roc_auc_SS = []

    for reps in range(nb_reps):
        xp_df, xu_df, yu_df = draw_p_u_dataset_scar('decaf_caltech', ds_ul, n_pos, n_unl, prior)
        xp = xp_df.to_numpy()
        xu = xu_df.to_numpy()
        yu = yu_df.to_numpy()

        costs, P, _, _ = ss_ipot(xp, xu, mu, nu, c, gamma=gamma_ss,max_outer_iter=100, max_inner_iter=50, return_map=True)
        nu_star = np.sum(P, axis=0)
        nu_star_sorted_idx = np.argsort(-nu_star)
        nu_start_selected_idx = nu_star_sorted_idx[0:int(prior_lst[i]*len(nu_star))]
        yu_hat = np.zeros_like(yu)
        yu_hat[nu_start_selected_idx] = 1
        acc_SS.append(np.mean(yu_hat == yu))
        # f1_scores_SS.append(f1_score(yu, yu_hat, average='binary'))
        # precision_SS_rep, recall_SS_rep, f1_score_SS_rep,  _ = precision_recall_fscore_support(yu, yu_hat, pos_label=1, average='binary')
        precision_SS_rep, recall_SS_rep, f1_score_SS_rep,  _ = precision_recall_fscore_support(yu, yu_hat, pos_label=1, average='binary')

        precision_SS.append(precision_SS_rep)
        recall_SS.append(recall_SS_rep)
        f1_scores_SS.append(f1_score_SS_rep)
        # roc_auc_SS.append(roc_auc_score(yu, yu_hat))
        roc_auc_SS.append(roc_auc_score(yu, nu_star))

    precision_SS_avg.append(np.mean(precision_SS))
    recall_SS_avg.append(np.mean(recall_SS))
    f1_scores_SS_avg.append(np.mean(f1_scores_SS))
    acc_SS_avg.append(np.mean(acc_SS))
    roc_auc_SS_avg.append(np.mean(roc_auc_SS))
    
    

    print("precision SS= ", precision_SS_avg[idx])
    print("recall SS= ", recall_SS_avg[idx])
    print("f1-score SS= ", f1_scores_SS_avg[idx])
    print("roc auc SS=", roc_auc_SS_avg[idx])
    print("acc_SS_avg", acc_SS_avg[idx])
    
    
    idx = idx+1


labeled=surf_Caltech10
unlabeled=surf_Caltech10
precision pW= 0.5
recall pW= 0.5
f1-score PW= 0.5
roc-auc PW = 0.8578333333333333
acc_pW_avg 0.9
precision pGW =  0.030000000000000006
recall pGW =  0.030000000000000006
f1-score PGW =  0.030000000000000006
roc-auc PGW = 0.4622222222222222
acc_pGW_avg 0.8720000000000001
precision SS=  0.5199999999999999
recall SS=  0.5199999999999999
f1-score SS=  0.52
roc auc SS= 0.7333333333333333
acc_SS_avg 0.9040000000000001
labeled=surf_Caltech10
unlabeled=surf_amazon
precision pW= 0.08
recall pW= 0.08
f1-score PW= 0.08000000000000002
roc-auc PW = 0.45455555555555555
acc_pW_avg 0.8160000000000002
precision pGW =  0.07
recall pGW =  0.07
f1-score PGW =  0.07
roc-auc PGW = 0.4763888888888889
acc_pGW_avg 0.868
precision SS=  0.08
recall SS=  0.08
f1-score SS=  0.08000000000000002
roc auc SS= 0.48888888888888893
acc_SS_avg 0.8160000000000001
labeled=surf_Caltech10
unlabeled=surf_webcam
precision pW= 0.11000000000000001
recall pW= 0.11000000000000001
f1-s

In [9]:
results_UCI = {'dataset': ['surf C -> surf C', 'surf C -> surf A', 'surf C -> surf W', 'surf C -> surf D', 'decaf C -> decaf C', 'decaf C -> decaf A', 'decaf C -> decaf W', 'decaf C -> decaf D'],
               '$\pi$': [0.1]*8,
               'acc PW': acc_pW_avg,
               'acc_PGW ': acc_pGW_avg,
               'acc SS': acc_SS_avg,
               'roc_auc_PW': roc_auc_pW_avg,
               'roc_auc_PGW': roc_auc_pGW_avg,
               'roc-auc SS': roc_auc_SS_avg
               
               }
results_UCI = pd.DataFrame(data=results_UCI)
results_UCI

,dataset,$\pi$,acc PW,acc_PGW,acc SS,roc_auc_PW,roc_auc_PGW,roc-auc SS
0,surf C -> surf C,0.1,0.900,0.872,0.904,0.857833,0.462222,0.733333
1,surf C -> surf A,0.1,0.816,0.868,0.816,0.454556,0.476389,0.488889
2,surf C -> surf W,0.1,0.822,0.860,0.822,0.470611,0.480667,0.505556
3,surf C -> surf D,0.1,0.800,0.870,0.800,0.375556,0.432833,0.444444
4,decaf C -> decaf C,0.1,0.940,0.862,0.944,0.949778,0.570889,0.968222
5,decaf C -> decaf A,0.1,0.802,0.882,0.804,0.398611,0.503056,0.456444
6,decaf C -> decaf W,0.1,0.802,0.886,0.800,0.429889,0.582722,0.596556
7,decaf C -> decaf D,0.1,0.808,0.922,0.804,0.454667,0.504222,0.453000


In [10]:
results_caltech_diff_domains = {'dataset': ['surf C -> surf C', 'surf C -> surf A', 'surf C -> surf W', 'surf C -> surf D', 'decaf C -> decaf C', 'decaf C -> decaf A', 'decaf C -> decaf W', 'decaf C -> decaf D'],
               '$\pi$': [0.1]*8,
               'acc pW': acc_pW_avg,
               'f1 pW ': f1_scores_pW_avg,
               'roc-auc pW':roc_auc_pW_avg,
               'acc pGW': acc_pGW_avg,
               'f1 pGW ': f1_scores_pGW_avg,
               'roc-auc pGW':roc_auc_pGW_avg,
               'acc SS': acc_SS_avg,
               'f1 SS ': f1_scores_SS_avg,
               'roc-auc SS':roc_auc_SS_avg,
              }
results_caltech_diff_domains = pd.DataFrame(data=results_caltech_diff_domains)
results_caltech_diff_domains

,dataset,$\pi$,acc pW,f1 pW,roc-auc pW,acc pGW,f1 pGW,roc-auc pGW,acc SS,f1 SS,roc-auc SS
0,surf C -> surf C,0.1,0.900,0.50,0.857833,0.872,0.03,0.462222,0.904,0.52,0.733333
1,surf C -> surf A,0.1,0.816,0.08,0.454556,0.868,0.07,0.476389,0.816,0.08,0.488889
2,surf C -> surf W,0.1,0.822,0.11,0.470611,0.860,0.07,0.480667,0.822,0.11,0.505556
3,surf C -> surf D,0.1,0.800,0.00,0.375556,0.870,0.02,0.432833,0.800,0.00,0.444444
4,decaf C -> decaf C,0.1,0.940,0.70,0.949778,0.862,0.20,0.570889,0.944,0.72,0.968222
5,decaf C -> decaf A,0.1,0.802,0.01,0.398611,0.882,0.11,0.503056,0.804,0.02,0.456444
6,decaf C -> decaf W,0.1,0.802,0.01,0.429889,0.886,0.26,0.582722,0.800,0.00,0.596556
7,decaf C -> decaf D,0.1,0.808,0.04,0.454667,0.922,0.13,0.504222,0.804,0.02,0.453000
